In [ ]:
from nbfix import load_catalog

from typing import Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from IPython.display import display
from matplotlib.ticker import AutoMinorLocator, MultipleLocator

pd.options.display.float_format = "{:,.2f}".format

In [ ]:
catalog = load_catalog()

df = pd.concat(
    [
        catalog.load("train.cycle"),
        catalog.load("train.clean"),
        catalog.load("train.pressure"),
        catalog.load("train.velocity"),
        catalog.load("train.rolling"),
        catalog.load("train.log"),
        catalog.load("train.sqrt"),
        catalog.load("train.model_mlp_out", version="2021-08-23T20.54.45.745Z").rename(
            columns={0: "pred1", "pa": "pred1"}
        ),
        catalog.load("train.model_mlp_out", version="2021-08-23T23.11.10.513Z").rename(
            columns={0: "pred2", "pa": "pred2"}
        ),
    ],
    axis=1,
)

display(df)


In [ ]:
def plot_cycle(n: int, figsize: Tuple[int, int] = (15, 5)):
    c = df[df["cycle"] == n]

    # get a multi-axes figure
    fig, ax1 = plt.subplots(figsize=figsize)
    fig.subplots_adjust(right=0.75)
    ax2, ax3, ax4 = ax1.twinx(), ax1.twinx(), ax1.twinx()
    ax3.spines.right.set_position(("axes", 1.07))
    ax4.spines.right.set_position(("axes", 1.14))

    # plot the data
    ax1.plot(c.phi, c.pa, c="lime", label="pressure", lw=4, alpha=0.6)
    # ax1.plot(c.phi, c.pred1, c="fuchsia", label="prediction 1", lw=2, alpha=0.6)
    ax1.plot(c.phi, c.pred2, c="fuchsia", label="prediction", lw=2, alpha=0.6)
    ax1.set_ylabel("pressure")

    ax2.plot(c.phi, c.acc, c="gray", label="acceleration", ls=":", lw=0, markersize=1, marker=".")
    ax2.set_ylabel("acceleration")

    ax3.plot(c.phi, c.vel_abs, c="goldenrod", ls=":", label="velocity (absolute)")
    ax3.set_ylabel("velocity (absolute)")

    # ax4.plot(c.phi, c.roll_25_std, c="fuchsia", ls="-", label="rolling std 25")
    # ax4.plot(c.phi, c.roll_50_std, c="coral", ls="-", label="rolling std 50")
    ax4.plot(c.phi, c.roll_100_std, c="olive", ls=":", label="ro'lling std 100")
    ax4.set_ylabel("rolling std")

    # place dots at maximum
    pa_max_xy = c.phi.iloc[c.pa.argmax()], c.pa.max()
    ax1.plot(*pa_max_xy, c="lime", marker="v", markersize=10, alpha=0.5)

    acc_max_xy = c.phi.iloc[c.acc.argmax()], c.acc.max()
    ax2.plot(*acc_max_xy, c="gray", marker="^", markersize=10, alpha=0.5)

    # grids
    ax1.xaxis.set_major_locator(MultipleLocator(30))
    ax1.xaxis.set_minor_locator(AutoMinorLocator(6))

    ax1.grid(which='major', color='#CCCCCC', linestyle='--')
    ax1.grid(which='minor', color='#CCCCCC', linestyle=':')
    
    # labels, titles
    ax1.set_xlabel("crank angle")
    ax1.set_title(f"{n}")

    fig.legend(loc="upper right", ncol=5)

    plt.tight_layout()
    plt.show()

In [ ]:
with plt.style.context("dark_background"):
    for i in np.random.randint(1, df["cycle"].max() + 1, size=30):
        plot_cycle(i)